In [9]:
import modulos as md
import numpy as np
import os
import sys
import constantes as cons
np.seterr(divide='ignore')
lista_filtrada = []
#lista=md.filtradodemonedas ()
lista = ['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'EOSUSDT', 'LTCUSDT', 'ETCUSDT', 'LINKUSDT', 'ADAUSDT', 'BNBUSDT', 'ATOMUSDT', 
         'ZRXUSDT', 'OMGUSDT', 'DOGEUSDT', 'WAVESUSDT', 'MKRUSDT', 'DOTUSDT', 'YFIUSDT', 'TRBUSDT', 'RUNEUSDT', 'SOLUSDT', 'UNIUSDT', 
         'AVAXUSDT', 'FTMUSDT', 'NEARUSDT', 'FILUSDT', 'MATICUSDT', 'DYDXUSDT', 'GALAUSDT', 'FLOWUSDT', 'GMTUSDT', 'APEUSDT', 'OPUSDT', 
         'INJUSDT', 'LDOUSDT', 'APTUSDT', 'ARBUSDT', 'SUIUSDT', 'WLDUSDT', 'ARKMUSDT', 'SEIUSDT', 'CYBERUSDT', 'ARKUSDT', 'GASUSDT', 
         'TIAUSDT', 'CAKEUSDT', 'MEMEUSDT', 'ORDIUSDT', 'NTRNUSDT']
lista = ['FTMUSDT']

timeframe = '15m'
if len(lista) > 1:
    imprimo = False
    debug = False
else:
    imprimo = True
    debug = True
print(lista)
porcentajes_sumados = 0
win_rate_buenos = 0
win_rate_malos = 0
trades = 0
balance = md.balancetotal()
for symbol in lista:    
    try:
        #data = md.estrategia_smart(symbol, debug = False, refinado = False, file_source = False, timeframe = timeframe, balance = balance, largo = 1)
        data = md.estrategia_alex(symbol, debug = False, refinado = False, file_source = False, timeframe = timeframe, balance = balance, largo = 6)
        resultado = md.backtesting_smart(data, plot_flag=imprimo, symbol=symbol)
        if resultado['Win Rate [%]'] >= 50:
            lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
        porcentajes_sumados=porcentajes_sumados+resultado['Return [%]']
        trades = trades + resultado['# Trades']
        if not np.isnan(resultado['Win Rate [%]']):
            if resultado['Win Rate [%]'] >= 50:
                win_rate_buenos = win_rate_buenos+1
            else:
                win_rate_malos = win_rate_malos+1
        if ((data.Close.iloc[-1] < (data.decisional_alcista_high.iloc[-2] + (data.atr.iloc[-1]*3)))
            or 
            (data.Close.iloc[-1] > (data.decisional_bajista_low.iloc[-2] - (data.atr.iloc[-1]*3)))
            ):
            print(f"posible entrada symbol: {symbol}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
print(f"\nTimeframe {timeframe} - porcentajes_sumados {md.truncate(porcentajes_sumados,2)} - trades {trades} - win_rate_buenos {win_rate_buenos} - win_rate_malos {win_rate_malos} - Ganancia por trade: {md.truncate((porcentajes_sumados/trades if trades !=0 else porcentajes_sumados),2)}%")


['FTMUSDT']


C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


FTMUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan

Timeframe 15m - porcentajes_sumados 0.0 - trades 0 - win_rate_buenos 0 - win_rate_malos 0 - Ganancia por trade: 0.0%


In [3]:
data

,Open Time,Open,High,Low,Close,Volume,ema20,ema50,ema200,atr,...,decisional_alcista,tendencia,trend,cruce_bos,sentido,signal,take_profit,stop_loss,cierra,porcentajeentrada
2023-11-12 08:10:00,2023-11-12 08:10:00,0.6670,0.6678,0.6652,0.6660,4232488.7,NaN,NaN,NaN,NaN,...,False,-2.0,Bajista,-2,-2,0,0.0,0.0,False,100
2023-11-12 08:15:00,2023-11-12 08:15:00,0.6660,0.6665,0.6642,0.6651,5483288.6,NaN,NaN,NaN,NaN,...,False,-2.0,Bajista,-2,-2,0,0.0,0.0,False,100
2023-11-12 08:20:00,2023-11-12 08:20:00,0.6651,0.6665,0.6646,0.6662,3062942.5,NaN,NaN,NaN,NaN,...,False,-2.0,Bajista,-2,-2,0,0.0,0.0,False,100
2023-11-12 08:25:00,2023-11-12 08:25:00,0.6662,0.6684,0.6662,0.6683,3566510.3,NaN,NaN,NaN,NaN,...,False,-2.0,Bajista,-2,-2,0,0.0,0.0,False,100
2023-11-12 08:30:00,2023-11-12 08:30:00,0.6684,0.6694,0.6679,0.6685,4435371.2,NaN,NaN,NaN,NaN,...,False,-2.0,Bajista,-2,-2,0,0.0,0.0,False,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-15 19:05:00,2023-11-15 19:05:00,0.6429,0.6460,0.6425,0.6431,12011293.0,0.638885,0.637735,0.636392,0.002880,...,False,-3.0,Bajista,-2,-2,0,0.0,0.0,False,100
2023-11-15 19:10:00,2023-11-15 19:10:00,0.6431,0.6436,0.6406,0.6410,10508870.0,0.639087,0.637863,0.636438,0.002889,...,False,-3.0,Bajista,-2,-2,0,0.0,0.0,False,100
2023-11-15 19:15:00,2023-11-15 19:15:00,0.6410,0.6413,0.6387,0.6404,7870151.2,0.639212,0.637962,0.636478,0.002868,...,False,-3.0,Bajista,-2,-2,0,0.0,0.0,False,100
2023-11-15 19:20:00,2023-11-15 19:20:00,0.6404,0.6416,0.6394,0.6413,3693998.6,0.639411,0.638093,0.636526,0.002821,...,False,-3.0,Bajista,-2,-2,0,0.0,0.0,False,100
